# Import Libraries

In [1]:
import pandas as pd
import warnings
warnings.simplefilter("ignore")

# Load Dataset
In this example, we use titanic dataset from OpenML.

In [2]:
from sklearn.datasets import fetch_openml

X, y = fetch_openml(data_id=40945, as_frame=True, return_X_y=True, parser='auto')

In [3]:
X.head()

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [4]:
X.isnull().sum()

pclass          0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64

In [5]:
X.dtypes

pclass          int64
name           object
sex          category
age           float64
sibsp           int64
parch           int64
ticket         object
fare          float64
cabin          object
embarked     category
boat           object
body          float64
home.dest      object
dtype: object

In [6]:
# Target is mapped to strings for later description
y = y.map({"0":"not survived", "1":"survived"})

In [7]:
y.value_counts()

survived
not survived    809
survived        500
Name: count, dtype: int64

In [8]:
y.isnull().sum()

0

In [9]:
y.dtypes

CategoricalDtype(categories=['not survived', 'survived'], ordered=False)

split dataset into train set and test set randomly

In [10]:
from sklearn.model_selection import train_test_split

X, X_test, y, y_test = train_test_split(X, y, shuffle=True, random_state=0, test_size=0.3)
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [11]:
X.head()

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,2,"Mellinger, Miss. Madeleine Violet",female,13.0,0,1,250644,19.5000,NaN,S,14,NaN,"England / Bennington, VT"
1,2,"Wells, Miss. Joan",female,4.0,1,1,29103,23.0000,NaN,S,14,NaN,"Cornwall / Akron, OH"
2,2,"Duran y More, Miss. Florentina",female,30.0,1,0,SC/PARIS 2148,13.8583,NaN,C,12,NaN,"Barcelona, Spain / Havana, Cuba"
3,3,"Scanlan, Mr. James",male,NaN,0,0,36209,7.7250,NaN,Q,NaN,NaN,NaN
4,3,"Bradley, Miss. Bridget Delia",female,22.0,0,0,334914,7.7250,NaN,Q,13,NaN,"Kingwilliamstown, Co Cork, Ireland Glens Falls..."


# Define Pipeline

## initiate pipeline

In [12]:
from imker import Pipeline, Task, TaskConfig, BaseTask, BaseProcessor, BaseSplitter, BaseModel
pipe = Pipeline(repo_dir="../../../cache", exp_name="example", pipeline_name="titanic")

## define preprocessing tasks

In this section, let's define TimeSleep task that just wait 2 seconds to show cache function of task, and set it to pipeline

In [13]:
from typing import Any, Union
import time

class DropCols(BaseTask):
    def __init__(self, cols:list) -> None:
        self.cols = cols
    
    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        X = X.drop(self.cols, axis=1)
        return X

class TimeSleep(BaseTask):
    def transform(self, X: Any) -> Any:
        time.sleep(1)
        return X

class DTypeConverter(BaseTask):
    def __init__(self, dtype: str):
        self.dtype = dtype

    def transform(self, X: Union[pd.DataFrame, pd.Series]) -> Union[pd.DataFrame, pd.Series]:
        X = X.astype(self.dtype)
        return X
    

## define PreProcessor using each tasks.
Task can accept only argument TaskConfig. Using Task class allows task to perform as like PyTorch layers. First call of task run fit method and transform method. Once task called, task run only transform method from second call.

In [14]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

class PreProcessor(BaseProcessor):
    def __init__(self):
        self.drop = Task(TaskConfig(task=DropCols, 
                                    init_params={"cols":["name", "cabin", "ticket", "body", "boat", "home.dest"]}, 
                                    ))
        self.cat_encoder = Task(TaskConfig(task=OrdinalEncoder, # you can use scikit learn class as it is
                                           init_params={"handle_unknown":"use_encoded_value", 
                                                        "unknown_value":-1, 
                                                        "encoded_missing_value":-999}, 
                                            ))
        self.target_label_enc = Task(TaskConfig(task=LabelEncoder))
        self.dtype_converter = Task(TaskConfig(task=DTypeConverter, 
                                               init_params={"dtype":"int8"}))
        self.sleep = Task(TaskConfig(task=TimeSleep, 
                                     cache=True))# If you pass True, output of the task will be cached.
        
    def forward(self, X, y=None):
        X = self.drop(X)
        X[["sex", "embarked"]] = self.cat_encoder(X[["sex", "embarked"]])
        X[["sex", "embarked"]] = self.dtype_converter(X[["sex", "embarked"]])
        if y is not None:
            y = self.target_label_enc(y) # target variable can be transformed as well as features
        X = self.sleep(X)
        return X, y


## set preprocessing to pipeline and see the results

In [15]:
pipe.set_preprocessor(PreProcessor)

In [16]:
%%time
pipe.test_preprocessing(X, y)

DropCols                       :             fit process takes 0.0074 [sec]
DropCols                       :       transform process takes 0.0011 [sec]
OrdinalEncoder                 :             fit process takes 0.0244 [sec]
OrdinalEncoder                 :       transform process takes 0.0014 [sec]
DTypeConverter                 :             fit process takes 0.0037 [sec]
DTypeConverter                 :       transform process takes 0.0004 [sec]
LabelEncoder                   :             fit process takes 0.0082 [sec]
LabelEncoder                   :       transform process takes 0.0005 [sec]
TimeSleep                      :             fit process takes 0.0023 [sec]
TimeSleep                      :       transform process takes 1.0071 [sec]
CPU times: user 59.4 ms, sys: 2.74 ms, total: 62.2 ms
Wall time: 1.06 s


(     pclass  sex      age  sibsp  parch     fare  embarked
 0         2    0  13.0000      0      1  19.5000         2
 1         2    0   4.0000      1      1  23.0000         2
 2         2    0  30.0000      1      0  13.8583         0
 3         3    1      NaN      0      0   7.7250         1
 4         3    0  22.0000      0      0   7.7250         1
 ..      ...  ...      ...    ...    ...      ...       ...
 911       3    0   0.1667      1      2  20.5750         2
 912       3    1      NaN      0      0   8.0500         2
 913       3    0      NaN      0      0   7.7333         1
 914       2    0  20.0000      0      0  36.7500         2
 915       3    0  32.0000      1      1  15.5000         1
 
 [916 rows x 7 columns],
 array([1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,
        0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
        1, 0, 1, 0, 0, 1, 0, 1,

TimeSleep task is cached, so it doesn't take a second to run from second time

In [17]:
%%time
pipe.test_preprocessing(X, y)

DropCols                       :             fit process takes 0.0028 [sec]
DropCols                       :       transform process takes 0.0012 [sec]
OrdinalEncoder                 :             fit process takes 0.0125 [sec]
OrdinalEncoder                 :       transform process takes 0.0017 [sec]
DTypeConverter                 :             fit process takes 0.0013 [sec]
DTypeConverter                 :       transform process takes 0.0005 [sec]
LabelEncoder                   :             fit process takes 0.0037 [sec]
LabelEncoder                   :       transform process takes 0.0006 [sec]
TimeSleep                      :             fit process takes 0.0009 [sec]
TimeSleep                      :       transform process takes 0.0020 [sec]
CPU times: user 32.7 ms, sys: 211 µs, total: 32.9 ms
Wall time: 30.9 ms


(     pclass  sex      age  sibsp  parch     fare  embarked
 0         2    0  13.0000      0      1  19.5000         2
 1         2    0   4.0000      1      1  23.0000         2
 2         2    0  30.0000      1      0  13.8583         0
 3         3    1      NaN      0      0   7.7250         1
 4         3    0  22.0000      0      0   7.7250         1
 ..      ...  ...      ...    ...    ...      ...       ...
 911       3    0   0.1667      1      2  20.5750         2
 912       3    1      NaN      0      0   8.0500         2
 913       3    0      NaN      0      0   7.7333         1
 914       2    0  20.0000      0      0  36.7500         2
 915       3    0  32.0000      1      1  15.5000         1
 
 [916 rows x 7 columns],
 array([1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,
        0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
        1, 0, 1, 0, 0, 1, 0, 1,

imker provides simple repository viewer to see outputs of each tasks.

In [18]:
from imker import RepositoryViewer
viewer = RepositoryViewer(repo_dir="../../../cache/")
viewer.search_repo()

,task_id,lastUpdatedDate,repo,method,processor,cachefile,config
0,5,2023-07-19T12:10:15.206610,../../../cache,transform,TimeSleep,../../../cache/task/transform/TimeSleep/51397c...,../../../cache/task/transform/TimeSleep/51397c...
1,4,2023-07-19T12:10:14.198541,../../../cache,fit,TimeSleep,../../../cache/task/fit/TimeSleep/e36debdeb673...,../../../cache/task/fit/TimeSleep/e36debdeb673...
2,3,2023-07-19T12:10:14.178540,../../../cache,fit,OrdinalEncoder,../../../cache/task/fit/OrdinalEncoder/0f9a614...,../../../cache/task/fit/OrdinalEncoder/0f9a614...
3,2,2023-07-19T12:10:14.194541,../../../cache,fit,LabelEncoder,../../../cache/task/fit/LabelEncoder/26a33793a...,../../../cache/task/fit/LabelEncoder/26a33793a...
4,1,2023-07-19T12:10:14.154538,../../../cache,fit,DropCols,../../../cache/task/fit/DropCols/074bd8c804115...,../../../cache/task/fit/DropCols/074bd8c804115...
5,0,2023-07-19T12:10:14.186540,../../../cache,fit,DTypeConverter,../../../cache/task/fit/DTypeConverter/ae1dc0e...,../../../cache/task/fit/DTypeConverter/ae1dc0e...


In [19]:
# load_cache method accepts int number of task_id or strings of path 
viewer.load_cache(5)

,pclass,sex,age,sibsp,parch,fare,embarked
0,2,0,13.0000,0,1,19.5000,2
1,2,0,4.0000,1,1,23.0000,2
2,2,0,30.0000,1,0,13.8583,0
3,3,1,NaN,0,0,7.7250,1
4,3,0,22.0000,0,0,7.7250,1
...,...,...,...,...,...,...,...
911,3,0,0.1667,1,2,20.5750,2
912,3,1,NaN,0,0,8.0500,2
913,3,0,NaN,0,0,7.7333,1
914,2,0,20.0000,0,0,36.7500,2


In [20]:
viewer.load_config(3)

{'init_params': {'encoded_missing_value': -999,
  'handle_unknown': 'use_encoded_value',
  'unknown_value': -1},
 'fit_params': {},
 'transform_params': {},
 'predict_params': {},
 'cache_processor': 'PickledBz2Cacher',
 'seed': 42}

## define data splitter for validation

In [21]:
from sklearn.model_selection import StratifiedKFold

class Splitter(BaseSplitter):
    def __init__(self):
        self.splitter = Task(TaskConfig(task=StratifiedKFold, 
                                    init_params={"n_splits":5, "shuffle":True}))

    def get_n_splits(self):
        return self.splitter.get_n_splits()
    
    def split(self, X, y=None):
        return self.splitter(X, y)

## set splitter to pipeline

In [22]:
pipe.set_splitter(Splitter)

## define oof preprocessing

In [23]:
class FillNa(BaseTask):
    def __init__(self, values:Union[dict, float]) -> None:
        super().__init__()
        self.values = values

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        return X.fillna(self.values)

In [24]:
from sklearn.preprocessing import StandardScaler

class OOFPreProcessor(BaseProcessor):
    def __init__(self):
        self.std_scaler = Task(TaskConfig(task=StandardScaler))
        self.fillna = Task(TaskConfig(task=FillNa, 
                                      init_params={"values":-999}))

    def forward(self, X, y=None):
        X[["age", "fare"]] = self.std_scaler(X[["age", "fare"]])
        X = self.fillna(X)
        return X, y

## set oof preprocessing to pipeline

In [25]:
pipe.set_oof_preprocessor(OOFPreProcessor)

## confirm oof subsets

In [26]:
g = pipe.test_oof_preprocessing(X, y)
oof = next(g)
oof.keys()

DropCols                       :             fit process takes 0.0035 [sec]
DropCols                       :       transform process takes 0.0014 [sec]
OrdinalEncoder                 :             fit process takes 0.0106 [sec]
OrdinalEncoder                 :       transform process takes 0.0014 [sec]
DTypeConverter                 :             fit process takes 0.0010 [sec]
DTypeConverter                 :       transform process takes 0.0005 [sec]
LabelEncoder                   :             fit process takes 0.0032 [sec]
LabelEncoder                   :       transform process takes 0.0006 [sec]
TimeSleep                      :             fit process takes 0.0008 [sec]
TimeSleep                      :       transform process takes 0.0017 [sec]
StratifiedKFold                :           split process takes 0.0000 [sec]
StandardScaler                 :             fit process takes 0.0297 [sec]
StandardScaler                 :       transform process takes 0.0014 [sec]
FillNa      

odict_keys(['X_train', 'y_train', 'X_valid', 'y_valid', 'idx_train', 'idx_valid'])

In [27]:
oof.X_train.head()

,pclass,sex,age,sibsp,parch,fare,embarked
0,2,0,-1.236365,0,1,-0.284444,2
2,2,0,-0.043607,1,0,-0.402318,0
3,3,1,-999.000000,0,0,-0.530462,1
4,3,0,-0.604905,0,0,-0.530462,1
5,3,1,-999.000000,0,0,-0.527240,1


In [28]:
oof.X_valid.head()

,pclass,sex,age,sibsp,parch,fare,embarked
1,2,0,-1.867825,1,1,-0.211318,2
6,1,0,-0.043607,0,0,0.497572,0
9,1,1,0.307204,0,0,-0.142632,2
10,2,1,-2.008149,1,1,-0.211318,2
13,2,1,-0.885554,0,0,-0.420250,2


## define model

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from lightgbm.callback import early_stopping
from lightgbm import LGBMClassifier

class Classifier(BaseModel):
    def __init__(self):
        self.lr = Task(TaskConfig(LogisticRegression))
        self.knn = Task(TaskConfig(KNeighborsClassifier))
        self.lgb = Task(TaskConfig(task=LGBMClassifier, 
                                   init_params={"n_estimators":200, 
                                                "n_jobs":1}, 
                                   fit_params={"callbacks":[
                                       early_stopping(stopping_rounds=30)
                                   ]}))

    def forward(self, X, y=None, eval_set=None, proba=False):
        return {"lr": self.lr(X, y, proba=proba), 
                "knn":self.knn(X, y, proba=proba), 
                "lgb":self.lgb(X, y, eval_set=eval_set, proba=proba)}

## set predictor to pipeline

In [30]:
pipe.set_model(Classifier)

## Train pipeline

In [31]:
pipe.train(X, y)

DropCols                       :             fit process takes 0.0033 [sec]
DropCols                       :       transform process takes 0.0014 [sec]
OrdinalEncoder                 :             fit process takes 0.0109 [sec]
OrdinalEncoder                 :       transform process takes 0.0014 [sec]
DTypeConverter                 :             fit process takes 0.0011 [sec]
DTypeConverter                 :       transform process takes 0.0005 [sec]
LabelEncoder                   :             fit process takes 0.0031 [sec]
LabelEncoder                   :       transform process takes 0.0006 [sec]
TimeSleep                      :             fit process takes 0.0008 [sec]
TimeSleep                      :       transform process takes 0.0022 [sec]
StratifiedKFold                :           split process takes 0.0000 [sec]
StandardScaler                 :             fit process takes 0.0113 [sec]
StandardScaler                 :       transform process takes 0.0017 [sec]
FillNa      

## validate with 2 metrics

In [32]:
from sklearn.metrics import accuracy_score, f1_score
pipe.set_metrics([accuracy_score, f1_score])

In [33]:
val_preds = pipe.validate(X, y)

DropCols                       :       transform process takes 0.0014 [sec]
OrdinalEncoder                 :       transform process takes 0.0022 [sec]
DTypeConverter                 :       transform process takes 0.0005 [sec]
LabelEncoder                   :       transform process takes 0.0005 [sec]
TimeSleep                      :       transform process takes 0.0020 [sec]
StratifiedKFold                :           split process takes 0.0000 [sec]
StandardScaler                 :       transform process takes 0.0016 [sec]
FillNa                         :       transform process takes 0.0004 [sec]
LogisticRegression             :         predict process takes 0.0017 [sec]
KNeighborsClassifier           :         predict process takes 0.0169 [sec]
LGBMClassifier                 :         predict process takes 0.0028 [sec]
StandardScaler                 :       transform process takes 0.0015 [sec]
FillNa                         :       transform process takes 0.0004 [sec]
LogisticRegr

In [34]:
pipe.get_scores()

fold0     fold1     fold2     fold3     fold4
lr  accuracy_score  0.793478  0.803279  0.765027  0.803279  0.792350
    f1_score        0.707692  0.739130  0.690647  0.727273  0.707692
knn accuracy_score  0.777174  0.786885  0.754098  0.737705  0.765027
    f1_score        0.672000  0.706767  0.689655  0.636364  0.676692
lgb accuracy_score  0.809783  0.841530  0.775956  0.814208  0.797814
    f1_score        0.724409  0.781955  0.677165  0.734375  0.708661

In [35]:
pd.DataFrame(val_preds)

,lr,knn,lgb
0,1,1,1
1,1,1,1
2,1,1,1
3,0,0,0
4,1,0,1
...,...,...,...
911,1,0,0
912,0,0,0
913,1,1,1
914,1,1,1


If you want to get the results of validation subsets as probability, you just specify the proba argument as True

In [36]:
val_preds = pipe.validate(X, y, proba=True, calc_metrics=False)

DropCols                       :       transform process takes 0.0011 [sec]
OrdinalEncoder                 :       transform process takes 0.0017 [sec]
DTypeConverter                 :       transform process takes 0.0004 [sec]
LabelEncoder                   :       transform process takes 0.0005 [sec]
TimeSleep                      :       transform process takes 0.0019 [sec]
StratifiedKFold                :           split process takes 0.0000 [sec]
StandardScaler                 :       transform process takes 0.0014 [sec]
FillNa                         :       transform process takes 0.0003 [sec]
LogisticRegression             :   predict_proba process takes 0.0017 [sec]
KNeighborsClassifier           :   predict_proba process takes 0.0028 [sec]
LGBMClassifier                 :   predict_proba process takes 0.0020 [sec]
StandardScaler                 :       transform process takes 0.0015 [sec]
FillNa                         :       transform process takes 0.0003 [sec]
LogisticRegr

In [37]:
val_preds.lr

array([[0.2167255 , 0.7832745 ],
       [0.27079821, 0.72920179],
       [0.23279805, 0.76720195],
       ...,
       [0.39257758, 0.60742242],
       [0.20470846, 0.79529154],
       [0.41247274, 0.58752726]])

## inference test data

In [38]:
test_preds = pipe.inference(X_test)

DropCols                       :       transform process takes 0.0012 [sec]
OrdinalEncoder                 :       transform process takes 0.0016 [sec]
DTypeConverter                 :       transform process takes 0.0005 [sec]
TimeSleep                      :       transform process takes 1.0062 [sec]
StandardScaler                 :       transform process takes 0.0017 [sec]
FillNa                         :       transform process takes 0.0004 [sec]
LogisticRegression             :         predict process takes 0.0016 [sec]
KNeighborsClassifier           :         predict process takes 0.0308 [sec]
LGBMClassifier                 :         predict process takes 0.0031 [sec]
StandardScaler                 :       transform process takes 0.0014 [sec]
FillNa                         :       transform process takes 0.0004 [sec]
LogisticRegression             :         predict process takes 0.0016 [sec]
KNeighborsClassifier           :         predict process takes 0.0307 [sec]
LGBMClassifi

In [39]:
test_preds.keys()

odict_keys(['lr', 'knn', 'lgb'])

In [40]:
test_preds.lgb[:30]

array([0. , 1. , 0. , 0. , 0. , 0.8, 0. , 0. , 0. , 0. , 0. , 0. , 0.6,
       0.2, 1. , 0. , 1. , 0. , 1. , 0. , 0. , 0. , 0. , 1. , 1. , 1. ,
       0. , 0. , 0. , 1. ])

Probability is easy to get

In [41]:
test_preds = pipe.inference(X_test, proba=True)

DropCols                       :       transform process takes 0.0011 [sec]
OrdinalEncoder                 :       transform process takes 0.0013 [sec]
DTypeConverter                 :       transform process takes 0.0004 [sec]
TimeSleep                      :       transform process takes 0.0017 [sec]
StandardScaler                 :       transform process takes 0.0015 [sec]
FillNa                         :       transform process takes 0.0004 [sec]
LogisticRegression             :   predict_proba process takes 0.0015 [sec]
KNeighborsClassifier           :   predict_proba process takes 0.0039 [sec]
LGBMClassifier                 :   predict_proba process takes 0.0024 [sec]
StandardScaler                 :       transform process takes 0.0015 [sec]
FillNa                         :       transform process takes 0.0004 [sec]
LogisticRegression             :   predict_proba process takes 0.0017 [sec]
KNeighborsClassifier           :   predict_proba process takes 0.0040 [sec]
LGBMClassifi

In [42]:
test_preds.lr[:30]

array([[0.85787877, 0.14212123],
       [0.2272882 , 0.7727118 ],
       [0.81224066, 0.18775934],
       [0.89553155, 0.10446845],
       [0.75813045, 0.24186955],
       [0.41949524, 0.58050476],
       [0.63938416, 0.36061584],
       [0.63636433, 0.36363567],
       [0.76922829, 0.23077171],
       [0.89762602, 0.10237398],
       [0.80608807, 0.19391193],
       [0.85593646, 0.14406354],
       [0.82761299, 0.17238701],
       [0.33818976, 0.66181024],
       [0.10991438, 0.89008562],
       [0.77087358, 0.22912642],
       [0.28178381, 0.71821619],
       [0.60078654, 0.39921346],
       [0.1201942 , 0.8798058 ],
       [0.8578293 , 0.1421707 ],
       [0.85820769, 0.14179231],
       [0.95568681, 0.04431319],
       [0.85792963, 0.14207037],
       [0.14646222, 0.85353778],
       [0.88100018, 0.11899982],
       [0.2210122 , 0.7789878 ],
       [0.77089763, 0.22910237],
       [0.83981604, 0.16018396],
       [0.88864185, 0.11135815],
       [0.33971494, 0.66028506]])

# Reproduce pipeline

In [43]:
pipe2 = Pipeline.load(repo_dir="../../../cache/", exp_name="example", pipeline_name="titanic", 
                      preprocessor=PreProcessor, 
                      splitter=Splitter,
                      oof_preprocessor=OOFPreProcessor, 
                      model=Classifier
                      )
pipe2

In [44]:
all(pipe2.inference(X_test).lr == pipe.inference(X_test).lr)

DropCols                       :       transform process takes 0.0012 [sec]
OrdinalEncoder                 :       transform process takes 0.0013 [sec]
DTypeConverter                 :       transform process takes 0.0004 [sec]
TimeSleep                      :       transform process takes 0.0015 [sec]
StandardScaler                 :       transform process takes 0.0016 [sec]
FillNa                         :       transform process takes 0.0004 [sec]
LogisticRegression             :         predict process takes 0.0016 [sec]
KNeighborsClassifier           :         predict process takes 0.0333 [sec]
LGBMClassifier                 :         predict process takes 0.0034 [sec]
StandardScaler                 :       transform process takes 0.0019 [sec]
FillNa                         :       transform process takes 0.0004 [sec]
LogisticRegression             :         predict process takes 0.0016 [sec]
KNeighborsClassifier           :         predict process takes 0.0354 [sec]
LGBMClassifi

True